In [1]:
import torch

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

seed = 0
# Set the seed for reproducibility
torch.manual_seed(seed)

import torch
import gpytorch
from botorch.models import SingleTaskGP
from gpytorch.likelihoods import FixedNoiseGaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import MaternKernel, ScaleKernel

# Example Usage for 1D
dim = 16

# Define your kernel
nu = 2.5
lengthscale = 0.1
outputscale = 1.0
base_kernel = MaternKernel(nu=nu).double()
base_kernel.lengthscale = torch.tensor([[lengthscale]])
scale_kernel = ScaleKernel(base_kernel).double()
scale_kernel.outputscale = torch.tensor([[outputscale]])

# Define Noise Level
noise_level = 1e-4

# Initialize Placeholder Data with Correct Dimensions
num_samples = 1  # Replace with actual number of samples
num_features = dim  # Replace with actual number of features
train_X = torch.empty(num_samples, num_features)  # Placeholder data
train_Y = torch.empty(num_samples, 1)             # Placeholder data
Yvar = torch.ones(num_samples) * noise_level

# Initialize Model
model = SingleTaskGP(train_X, train_Y, likelihood = FixedNoiseGaussianLikelihood(noise=Yvar), covar_module=scale_kernel)

from botorch.sampling.pathwise import draw_kernel_feature_paths
matern_sample = draw_kernel_feature_paths(model, sample_shape=torch.Size([1]))
def objective_function(x):
    return matern_sample(x).squeeze(0).detach()

from botorch.utils.sampling import optimize_posterior_samples
# Find the global optimum
maximize = True
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
global_optimum_point, global_optimum_value = optimize_posterior_samples(paths=matern_sample, bounds=bounds, raw_samples=1024*dim, num_restarts=20*dim, maximize=maximize)

from gpytorch.kernels import MaternKernel, ScaleKernel
# Set up the kernel
base_kernel = MaternKernel(nu=nu).double()
base_kernel.lengthscale = lengthscale
base_kernel.raw_lengthscale.requires_grad = False
scale_kernel = ScaleKernel(base_kernel).double()
scale_kernel.outputscale = torch.tensor([[outputscale]])
scale_kernel.raw_outputscale.requires_grad = False

maximize = True
output_standardize = False

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-2.]

In [2]:
from botorch.utils.sampling import draw_sobol_samples
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from pandora_bayesopt.acquisition.stable_gittins import LogGittinsIndex
num_iterations = 2
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
LogPBGI_optimizer = BayesianOptimizer( 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x,
        objective=objective_function, 
        output_standardize=output_standardize
    )
# Run the optimization
LogPBGI_optimizer.run(
            num_iterations = num_iterations, 
            acquisition_function_class = LogGittinsIndex,
            lmbda = 0.0001
        )

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.2982]), std = tensor([0.9299])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 0, New point: [0.         0.27417155 0.7741499  0.02928046], New value: -1.3433043385822367
Best observed value: 0.799035898007519
Current acquisition value: 3.2872643309805145
Cumulative cost: 1.0
Running time: 8.120061999999999



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.3932]), std = tensor([0.9368])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 1, New point: [0.97264644 0.88428744 0.80060517 0.67955449], New value: 0.09134786346977403
Best observed value: 0.799035898007519
Current acquisition value: 3.116960570132031
Cumulative cost: 2.0
Running time: 9.721238



In [2]:
from botorch.utils.sampling import draw_sobol_samples
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from pandora_bayesopt.acquisition.gittins import LogVanillaGittinsIndex
num_iterations = 2
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
LogVanillaPBGI_optimizer = BayesianOptimizer( 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x,
        objective=objective_function, 
        output_standardize=output_standardize
    )
# Run the optimization
LogVanillaPBGI_optimizer.run(
            num_iterations = num_iterations, 
            acquisition_function_class = LogVanillaGittinsIndex,
            lmbda = 0.0001
        )

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.2982]), std = tensor([0.9299])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 0, New point: [0.         0.27417155 0.7741499  0.02928046], New value: -1.3433043385821861
Best observed value: 0.799035898007519
Current acquisition value: 3.287264330980509
Cumulative cost: 1.0
Running time: 3.7586699999999986



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.3932]), std = tensor([0.9368])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 1, New point: [0.97264644 0.88428744 0.80060517 0.67955449], New value: 0.09134786351958663
Best observed value: 0.799035898007519
Current acquisition value: 3.116960570132016
Cumulative cost: 2.0
Running time: 4.349054000000001



In [2]:
from botorch.utils.sampling import draw_sobol_samples
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from pandora_bayesopt.acquisition.stable_gittins import RobustGittinsIndex
num_iterations = 2
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
RobustPBGI_optimizer = BayesianOptimizer( 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x,
        objective=objective_function, 
        output_standardize=output_standardize
    )
# Run the optimization
RobustPBGI_optimizer.run(
            num_iterations = num_iterations, 
            acquisition_function_class = RobustGittinsIndex,
            lmbda = 0.0001
        )

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.2982]), std = tensor([0.9299])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 0, New point: [0.         0.27414241 0.77417126 0.02928116], New value: -1.3432093708360693
Best observed value: 0.799035898007519
Current acquisition value: 3.287264330692003
Cumulative cost: 1.0
Running time: 10.250326999999999



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.3932]), std = tensor([0.9368])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 1, New point: [0.97261306 0.88430817 0.8006507  0.67954832], New value: 0.09211499885115043
Best observed value: 0.799035898007519
Current acquisition value: 3.1169288839850147
Cumulative cost: 2.0
Running time: 11.521773



In [2]:
from botorch.utils.sampling import draw_sobol_samples
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from pandora_bayesopt.acquisition.gittins import GittinsIndex
num_iterations = 2
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
PBGI_optimizer = BayesianOptimizer( 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x,
        objective=objective_function, 
        output_standardize=output_standardize
    )
# Run the optimization
PBGI_optimizer.run(
            num_iterations = num_iterations, 
            acquisition_function_class = GittinsIndex,
            lmbda = 0.0001
        )
PBGI_best_history = PBGI_optimizer.get_best_history()

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.0604]), std = tensor([0.9419])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 0, New point: [0.24381135 0.29459949 0.13814356 0.34680617 0.48369523 0.33734348
 0.07876036 0.5354155  0.1558262  0.45535102 0.52425464 0.09485292
 0.91282026 0.71975069 0.34654306 0.93534072], New value: -1.177566154869577
Best observed value: 1.7826202828511388
Current acquisition value: 3.6182402817000687
Cumulative cost: 1.0
Running time: 34.578129000000004



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOpt/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([-0.0923]), std = tensor([0.9469])). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Iteration 1, New point: [0.16334457 0.30589237 0.         0.28729548 0.34783276 0.27995139
 0.1941246  0.62273797 0.16048581 0.65909928 0.84886096 0.30901299
 0.95239987 0.69477847 0.30311739 0.87845621], New value: 1.8079968250560587
Best observed value: 1.8079968250560587
Current acquisition value: 3.710353374090138
Cumulative cost: 2.0
Running time: 44.52635000000001

